# Tutorial 1: `GenomicRanges` and range-based analyses

Genomic range operations are fundamental to many bioinformatics analyses. They allow us to work with intervals of genomic coordinates, which is crucial for understanding the relationships between different genomic features such as genes, regulatory elements, and experimental data like ChIP-seq peaks. In this tutorial, we'll explore how to work with genomic interval data using BiocPy's [GenomicRanges](https://github.com/BiocPy/GenomicRanges/) package, which provides Python implementations similar to the R/Bioconductor [GenomicRanges package](https://github.com/Bioconductor/GenomicRanges).

## Outline

In this workshop, we'll walk through several key aspects of working with genomic ranges in Python:

1. Reading Genomic Data: We'll start by reading in genomic data from RDS files, including exon positions grouped by transcripts.
2. Basic Genomic Operations: We'll cover fundamental operations like finding transcription start sites (TSS) and promoter regions.
3. Overlap Analysis: We'll learn how to find overlaps between different genomic features, a common task in many analyses.
4. Advanced Operations: We'll explore more complex operations like finding peaks within specific regions and resizing genomic intervals.

## Prerequisites

Before we begin, please ensure that you have the following packages installed:

### Installation

Let's start by installing the required packages for R and Python.

#### Python (shell)

You can install the Python packages using pip:

In [22]:
!pip install -U biocutils genomicranges rds2py numpy pandas geniml

  Using cached numpy-2.0.0-cp310-cp310-macosx_14_0_arm64.whl.metadata (60 kB)


#### R

    ```r
    BiocManager::install(c("AnnotationHub"), 
         repos='http://cran.us.r-project.org')
    ```

## 1. Save Annotations as RDS

Let's download the human reference genome and save the exon positions grouped by transcripts. For the purpose of the tutorial, we'll limit the exons to chromosome 22.

#### R

    ```r
    suppressMessages(library(AnnotationHub))
    
    ah <- AnnotationHub()
    ensdb <- query(ah, "Ensembl 112 EnsDb for Homo sapiens")[[1]]
    exons_by_tx <- exonsBy(ensdb, 
        by = "tx", filter = SeqNameFilter(c("22")), 
        columns= c("exon_id", "tx_name", "tx_id", "gene_name", "gene_id"))
    saveRDS(exons_by_tx, "hg38_exons_by_tx.rds")
    ```

## 2. Reading RDS files in Python

The [rds2py](https://github.com/biocpy/rds2py) Python package allows us to read RDS files and create equivalent Python representations of R objects. Key features include:

- Parsing common R objects into Python equivalents (e.g., matrices to NumPy arrays, data frames to Pandas DataFrames).
- Ability to read S4 classes, enabling direct parsing of Bioconductor data types from R to Python.

Reading an RDS file with rds2py involves two steps:

1. Parse the RDS file into a Python dictionary containing data, its attributes, and associated metadata.
2. Convert this dictionary into a suitable Python object using specific parser functions.

This process allows a seamless transition between R and Python for bioinformatics analyses.

In [23]:
from rds2py import read_rds
hg38_robject = read_rds("./hg38_exons_by_tx.rds")

from rds2py.granges import as_granges_list
by_tx = as_granges_list(hg38_robject)

print("Exons by transcript:")
print(by_tx)

Exons by transcript:
GenomicRangesList with 5387 ranges and 0 metadata columns
 
Name: ENST00000006251 
GenomicRanges with 9 ranges and 6 metadata columns
    seqnames              ranges          strand           exon_id         tx_name           tx_id gene_name         gene_id exon_rank
       <str>           <IRanges> <ndarray[int8]>            <list>          <list>          <list>    <list>          <list>    <list>
[0]    chr22 44677057 - 44677241               + | ENSE00001838743 ENST00000006251 ENST00000006251      PRR5 ENSG00000186654         1
[1]    chr22 44702492 - 44702609               + | ENSE00003647870 ENST00000006251 ENST00000006251      PRR5 ENSG00000186654         2
[2]    chr22 44714591 - 44714672               + | ENSE00003614159 ENST00000006251 ENST00000006251      PRR5 ENSG00000186654         3
[3]    chr22 44725244 - 44725293               + | ENSE00003568825 ENST00000006251 ENST00000006251      PRR5 ENSG00000186654         4
[4]    chr22 44726577 - 44726635   

## 2. Basic Genomic Operations

Now, let's perform some basic operations like finding Transcription Start Sites (TSS) and promoter regions. These operations are fundamental in genomic analysis as they help us identify key regulatory regions of the genome.

### 2.1 Create a `GenomicRangesList` by gene

To identify TSS or define promoter regions, let's first reprocess the input to create a `GenomicRangesList` by gene symbols.

To achieve this, we unlist the `GenomicRangesList` object. This is accomplished in Python using the `as_genomic_ranges()` method. 

In [24]:
all_ranges = by_tx.as_genomic_ranges()

Then we split the object using the `gene_name` metadata column in `mcols()`. 

```{important}
We provide accessors to get or set attributes of the class. Most folks in Python may be familiar with direct access to class members (via properties or @property), but this should generally be avoided, as it is too easy to perform modifications via one-liners with the class.property on the left-hand side of an assignment.

For more information, please refer to our [developer guide](https://github.com/BiocPy/developer_guide).
```

```{note}
While gene IDs are unique, gene symbols are not. In addition, this list has genes with no symbols.
```

In [25]:
by_gene = all_ranges.split(
    groups=all_ranges.get_mcols().get_column("gene_name")
)

print("Exons by gene:")
print(by_gene)

Exons by gene:
GenomicRangesList with 932 ranges and 0 metadata columns
 
Name:  
GenomicRanges with 1846 ranges and 6 metadata columns
                seqnames              ranges          strand           exon_id         tx_name           tx_id gene_name         gene_id exon_rank
                   <str>           <IRanges> <ndarray[int8]>            <list>          <list>          <list>    <list>          <list>    <list>
ENST00000255890    chr22 23402361 - 23402568               - | ENSE00001754779 ENST00000255890 ENST00000255890           ENSG00000290920         1
ENST00000255890    chr22 23402037 - 23402156               - | ENSE00001700441 ENST00000255890 ENST00000255890           ENSG00000290920         2
ENST00000255890    chr22 23401841 - 23401954               - | ENSE00004028898 ENST00000255890 ENST00000255890           ENSG00000290920         3
                     ...                 ...             ... |             ...             ...             ...       ...         

### 2.2 Finding Transcription Start Sites (TSS)

Transcription Start Sites (TSS) are the locations where transcription of a gene begins. Identifying TSS is crucial for understanding gene regulation, as many regulatory elements are located near the TSS. 

Let's use the `range()` method to get the full extent of each gene.

In [26]:
ranges_by_gene = by_gene.range()

print("Gene ranges:")
print(ranges_by_gene)

Gene ranges:
GenomicRangesList with 932 ranges and 0 metadata columns
 
Name:  
GenomicRanges with 2 ranges and 0 metadata columns
    seqnames              ranges          strand
       <str>           <IRanges> <ndarray[int8]>
[0]    chr22 11066418 - 50674175               +
[1]    chr22 15282557 - 50755435               -
------
seqinfo(1 sequences): chr22
 
Name: 5_8S_rRNA 
GenomicRanges with 1 range and 0 metadata columns
    seqnames              ranges          strand
       <str>           <IRanges> <ndarray[int8]>
[0]    chr22 11249809 - 11249960               -
------
seqinfo(1 sequences): chr22
 
Name: A4GALT 
GenomicRanges with 1 range and 0 metadata columns
    seqnames              ranges          strand
       <str>           <IRanges> <ndarray[int8]>
[0]    chr22 42692121 - 42721299               -
------
seqinfo(1 sequences): chr22
 
Name:  
GenomicRanges with 1 range and 0 metadata columns
    seqnames              ranges          strand
       <str>           <IRange

We convert the list to a `GenomicRanges` object.

In [27]:
gr_by_gene = ranges_by_gene.as_genomic_ranges()

print("as GenomicRanges:")
print(gr_by_gene)

as GenomicRanges:
GenomicRanges with 936 ranges and 0 metadata columns
          seqnames              ranges          strand
             <str>           <IRanges> <ndarray[int8]>
             chr22 11066418 - 50674175               +
             chr22 15282557 - 50755435               -
5_8S_rRNA    chr22 11249809 - 11249960               -
               ...                 ...             ...
    ZNRF3    chr22 28883572 - 29057489               +
ZNRF3-AS1    chr22 29024999 - 29031477               -
ZNRF3-IT1    chr22 28992721 - 29018621               +
------
seqinfo(1 sequences): chr22


Then we resize to a width of 1 base pair at the start of each range to pinpoint the TSS.

In [28]:
tss = gr_by_gene.resize(width=1, fix="start")

print("Transcript Start Sites:")
print(gr_by_gene)

Transcript Start Sites:
GenomicRanges with 936 ranges and 0 metadata columns
          seqnames              ranges          strand
             <str>           <IRanges> <ndarray[int8]>
             chr22 11066418 - 50674175               +
             chr22 15282557 - 50755435               -
5_8S_rRNA    chr22 11249809 - 11249960               -
               ...                 ...             ...
    ZNRF3    chr22 28883572 - 29057489               +
ZNRF3-AS1    chr22 29024999 - 29031477               -
ZNRF3-IT1    chr22 28992721 - 29018621               +
------
seqinfo(1 sequences): chr22


### 2.3 Defining Promoter Regions

Here, we're defining promoters as the region 2000 base pairs upstream to 200 base pairs downstream of each TSS. This definition can vary depending on the specific analysis, but this range often captures important regulatory elements.

In [29]:
promoters = tss.promoters(upstream=2000, downstream=200)

print("Promoter Regions:")
print(promoters)

Promoter Regions:
GenomicRanges with 936 ranges and 0 metadata columns
          seqnames              ranges          strand
             <str>           <IRanges> <ndarray[int8]>
             chr22 11064418 - 11066618               +
             chr22 50755235 - 50757435               -
5_8S_rRNA    chr22 11249760 - 11251960               -
               ...                 ...             ...
    ZNRF3    chr22 28881572 - 28883772               +
ZNRF3-AS1    chr22 29031277 - 29033477               -
ZNRF3-IT1    chr22 28990721 - 28992921               +
------
seqinfo(1 sequences): chr22


```{note}
Please be aware that because gene symbols may not be unique, this `GenomicRanges` object might contain duplicates. You might want to resolve duplicate symbols by making the symbols unique. We will leave this as an exercise for the reader.
```

## 3. Overlap Analysis

A common task in genomic analysis is finding overlaps between different genomic features. This helps us understand the relationships between various elements in the genome and can provide insights into gene regulation and function.

### 3.1 Reading ChIP-seq Peaks

ChIP-seq (Chromatin Immunoprecipitation followed by sequencing) is a method used to identify binding sites of DNA-associated proteins. The peaks represent regions where a protein of interest is likely bound to the DNA. We're focusing on chromosome 22 for this example to keep the dataset manageable.

For the purpose of this tutorial, let's download a bed file containing peaks from a ChIP-seq experiment on "Human B cells" to identify "EZH2" binding sites (from ENCODE) and cataloged in [bedbase.org](https://bedbase.org/bed/be4054acf6e3feeb4dc490e6430e358e).

In [30]:
from geniml.bbclient import BBClient

bbclient = BBClient(cache_folder="cache", bedbase_api="https://api.bedbase.org")
bedfile_id = "be4054acf6e3feeb4dc490e6430e358e" 
bedfile = bbclient.load_bed(bedfile_id)
peaks = bedfile.to_granges()

filter_chr22 = [x == "chr22" for x in peaks.get_seqnames()]
peaks_chr22 = peaks[filter_chr22]

print(peaks_chr22)

GenomicRanges with 1441 ranges and 0 metadata columns
       seqnames              ranges          strand
          <str>           <IRanges> <ndarray[int8]>
   [0]    chr22 19766788 - 19767078               *
   [1]    chr22 17369888 - 17370178               *
   [2]    chr22 19756445 - 19756735               *
            ...                 ...             ...
[1438]    chr22 27212058 - 27212348               *
[1439]    chr22 49201359 - 49201649               *
[1440]    chr22 49663362 - 49663652               *
------
seqinfo(46 sequences): chr1 chr10 chr11 ... chrUn_KI270750v1 chrUn_KI270752v1 chrX


### 3.2 Finding Overlaps with TSS

Here, we're identifying ChIP-seq peaks that overlap with TSS. This analysis can help us understand if the protein of interest tends to bind near the start of genes, which could suggest a role in transcription initiation.

In [31]:
overlaps = peaks_chr22.find_overlaps(tss)

print("Peak indices that overlap with first 10 TSS:")
print(overlaps[:10])

Peak indices that overlap with first 10 TSS:
[[], [], [], [55], [217], [], [], [], [], []]


```{note}
`find_overlaps` returns a `list` with the same length as TSS, indicating which indices from peaks overlap with each of the TSS. Ideally, we would want to return a `Hits` object similar to the Bioconductor implementation.

**TODO: Future plans to convert this into a `Hits` object.**
```

Let's identify the peaks that overlap with TSS.

In [32]:
import itertools

all_indices = list(set(itertools.chain.from_iterable(overlaps)))
peaks_by_tss = peaks_chr22[all_indices]
print(peaks_by_tss)

GenomicRanges with 35 ranges and 0 metadata columns
     seqnames              ranges          strand
        <str>           <IRanges> <ndarray[int8]>
 [0]    chr22 19756445 - 19756735               *
 [1]    chr22 38467935 - 38468225               *
 [2]    chr22 24952664 - 24952954               *
          ...                 ...             ...
[32]    chr22 21032552 - 21032842               *
[33]    chr22 50270553 - 50270843               *
[34]    chr22 19131257 - 19131547               *
------
seqinfo(46 sequences): chr1 chr10 chr11 ... chrUn_KI270750v1 chrUn_KI270752v1 chrX


Instead, one can subset peaks that overlap with TSS using the `subset_by_overlaps` method:

In [33]:
peaks_by_tss2 = peaks_chr22.subset_by_overlaps(tss)
print(peaks_by_tss2)

GenomicRanges with 35 ranges and 0 metadata columns
     seqnames              ranges          strand
        <str>           <IRanges> <ndarray[int8]>
 [0]    chr22 19756445 - 19756735               *
 [1]    chr22 38467935 - 38468225               *
 [2]    chr22 24952664 - 24952954               *
          ...                 ...             ...
[32]    chr22 21032552 - 21032842               *
[33]    chr22 50270553 - 50270843               *
[34]    chr22 19131257 - 19131547               *
------
seqinfo(46 sequences): chr1 chr10 chr11 ... chrUn_KI270750v1 chrUn_KI270752v1 chrX


Additionally, in some cases, we may want to ignore strand information (`ignore_strand=True`) when finding overlaps.

In [34]:
peaks_by_tss_ignoring_strand = peaks_chr22.subset_by_overlaps(tss, ignore_strand=True)
print(peaks_by_tss_ignoring_strand)

GenomicRanges with 35 ranges and 0 metadata columns
     seqnames              ranges          strand
        <str>           <IRanges> <ndarray[int8]>
 [0]    chr22 19756445 - 19756735               *
 [1]    chr22 38467935 - 38468225               *
 [2]    chr22 24952664 - 24952954               *
          ...                 ...             ...
[32]    chr22 21032552 - 21032842               *
[33]    chr22 50270553 - 50270843               *
[34]    chr22 19131257 - 19131547               *
------
seqinfo(46 sequences): chr1 chr10 chr11 ... chrUn_KI270750v1 chrUn_KI270752v1 chrX


```{note}
This yields the same results for this particular scenario, but may not if the 'peaks' contain strand information.
```

### 3.3 Finding Overlaps with Promoters

This operation finds ChIP-seq peaks that overlap with our defined promoter regions. If a significant number of peaks fall within promoters, it might suggest that the protein plays a role in gene regulation through promoter binding. This kind of analysis is often used to characterize the binding patterns of transcription factors or other regulatory proteins.

In [35]:
peaks_by_promoters = peaks_chr22.subset_by_overlaps(promoters)

print("Peaks Overlapping with Promoters:")
print(peaks_by_promoters)

Peaks Overlapping with Promoters:
GenomicRanges with 190 ranges and 0 metadata columns
      seqnames              ranges          strand
         <str>           <IRanges> <ndarray[int8]>
  [0]    chr22 19756445 - 19756735               *
  [1]    chr22 37427967 - 37428257               *
  [2]    chr22 22521942 - 22522232               *
           ...                 ...             ...
[187]    chr22 39993439 - 39993729               *
[188]    chr22 22338004 - 22338294               *
[189]    chr22 19131257 - 19131547               *
------
seqinfo(46 sequences): chr1 chr10 chr11 ... chrUn_KI270750v1 chrUn_KI270752v1 chrX


### 3.4 Finding Overlaps with Exons

Another analysis is to look at overlaps with all exons. This can help identify potential roles of the ChIP-seq peaks in splicing. Let's modify our analysis to look at all exons:

In [36]:
# Combine all exons into a single GenomicRanges object
all_exons = by_gene.as_granges()

# Find peaks overlapping with any exon
peaks_by_exons = peaks_chr22.subset_by_overlaps(all_exons)

print("Peaks overlapping with exons:")
print(peaks_by_exons)

# Calculate the percentage of peaks that overlap with exons
percent_overlapping = (len(peaks_by_exons) / len(peaks_chr22)) * 100

print(f"Percentage of peaks overlapping with exons: {percent_overlapping:.2f}%")

Peaks overlapping with exons:
GenomicRanges with 279 ranges and 0 metadata columns
      seqnames              ranges          strand
         <str>           <IRanges> <ndarray[int8]>
  [0]    chr22 19766788 - 19767078               *
  [1]    chr22 17369888 - 17370178               *
  [2]    chr22 29307104 - 29307394               *
           ...                 ...             ...
[276]    chr22 16969920 - 16970210               *
[277]    chr22 35552420 - 35552710               *
[278]    chr22 37931897 - 37932187               *
------
seqinfo(46 sequences): chr1 chr10 chr11 ... chrUn_KI270750v1 chrUn_KI270752v1 chrX
Percentage of peaks overlapping with exons: 19.36%


This analysis can provide insights into whether the protein of interest (captured by the ChIP-seq: "EZH2") tends to bind within gene bodies, potentially influencing gene expression, splicing, or other co-transcriptional processes.

## 4. Advanced Operations

Let's explore some more complex operations that are often used in genomic analyses.

### 4.1 Comparing Exonic vs. Intronic Binding

Let's first identify intron regions. We will use the `by_gene` object we created that contains a `GenomicRangesList` split by gene.

In [37]:
# Create intronic regions (regions within genes but not in exons)
gene_ranges = by_gene.range().as_genomic_ranges()  # Get the full extent of each gene
introns = gene_ranges.subtract(all_exons).as_granges()

print("Intron regions:")
print(introns)

Intron regions:
GenomicRanges with 1572 ranges and 0 metadata columns
          seqnames              ranges          strand
             <str>           <IRanges> <ndarray[int8]>
             chr22 15282557 - 15550904               -
             chr22 15552888 - 15553211               -
             chr22 15553587 - 15553691               -
               ...                 ...             ...
    ZNRF3    chr22 28883572 - 29057489               +
ZNRF3-AS1    chr22 29024999 - 29031477               -
ZNRF3-IT1    chr22 28992721 - 29018621               +
------
seqinfo(1 sequences): chr22


To gain further insight, we can compare the proportion of peaks overlapping with exons to those overlapping with introns:

In [38]:
# Find peaks overlapping with introns
peaks_by_introns = peaks_chr22.subset_by_overlaps(introns)

print("Peaks overlapping with introns:")
print(peaks_by_introns)

# Calculate percentages
percent_exonic = (len(peaks_by_exons) / len(peaks_chr22)) * 100
percent_intronic = (len(peaks_by_introns) / len(peaks_chr22)) * 100

print(f"Percentage of peaks overlapping with exons: {percent_exonic:.2f}%")
print(f"Percentage of peaks overlapping with introns: {percent_intronic:.2f}%")

Peaks overlapping with introns:
GenomicRanges with 1438 ranges and 0 metadata columns
       seqnames              ranges          strand
          <str>           <IRanges> <ndarray[int8]>
   [0]    chr22 19766788 - 19767078               *
   [1]    chr22 17369888 - 17370178               *
   [2]    chr22 19756445 - 19756735               *
            ...                 ...             ...
[1435]    chr22 27212058 - 27212348               *
[1436]    chr22 49201359 - 49201649               *
[1437]    chr22 49663362 - 49663652               *
------
seqinfo(46 sequences): chr1 chr10 chr11 ... chrUn_KI270750v1 chrUn_KI270752v1 chrX
Percentage of peaks overlapping with exons: 19.36%
Percentage of peaks overlapping with introns: 99.79%


```{note}
These percentages add up to over 100% because some peaks overlap both introns and exons, depending on how wide the peaks are. Ideally, you may want to filter the peaks based on preference as you annotate them with TSS, promoters, etc.
```

This comparison can help determine if the protein of interest shows a preference for binding in exonic or intronic regions, which could suggest different functional roles (e.g., splicing regulation for exonic binding vs. potential enhancer activity for intronic binding).

### 4.2 Finding Overlaps with the first exon

```{note}
- This analysis is performed by transcript.
- The rationale for this analysis may vary, but we are mostly showcasing complex genomic operations that are possible with the package.
```

Let's first put together a `GenomicRanges` object containing the first exon for each transcript.

In [39]:
all_first = []
for txid, grl in by_tx:
    strand = grl.get_strand(as_type = "list")[0]
    if strand == "-":
        all_first.append(grl.sort()[-1])
    else:
        all_first.append(grl.sort()[0])

Then we combine all the individual genomic elements. The [biocutils](https://github.com/BiocPy/BiocUtils) package provides utilities for convenient aspects of R that aren't provided by base Python and generics. One of these generics is the `'combine'` operation that merges or concatenates various Bioconductor classes.

In [40]:
from biocutils import combine_sequences
first_exons = combine_sequences(*all_first)

We can now subset peaks that overlap with the first exon

In [41]:
peaks_with_first_exons = peaks_chr22.subset_by_overlaps(first_exons)
print(peaks_with_first_exons)

GenomicRanges with 153 ranges and 0 metadata columns
      seqnames              ranges          strand
         <str>           <IRanges> <ndarray[int8]>
  [0]    chr22 17369888 - 17370178               *
  [1]    chr22 19756445 - 19756735               *
  [2]    chr22 45975507 - 45975797               *
           ...                 ...             ...
[150]    chr22 49500975 - 49501265               *
[151]    chr22 19131257 - 19131547               *
[152]    chr22 29307104 - 29307394               *
------
seqinfo(46 sequences): chr1 chr10 chr11 ... chrUn_KI270750v1 chrUn_KI270752v1 chrX


### 4.3 Resizing and Shifting Peaks

In [42]:
narrow_peaks = peaks_chr22.narrow(start=10, width=100)
shifted_peaks = narrow_peaks.shift(10)

print("Narrowed and Shifted Peaks:")
print(shifted_peaks)

Narrowed and Shifted Peaks:
GenomicRanges with 1441 ranges and 0 metadata columns
       seqnames              ranges          strand
          <str>           <IRanges> <ndarray[int8]>
   [0]    chr22 19766807 - 19766907               *
   [1]    chr22 17369907 - 17370007               *
   [2]    chr22 19756464 - 19756564               *
            ...                 ...             ...
[1438]    chr22 27212077 - 27212177               *
[1439]    chr22 49201378 - 49201478               *
[1440]    chr22 49663381 - 49663481               *
------
seqinfo(46 sequences): chr1 chr10 chr11 ... chrUn_KI270750v1 chrUn_KI270752v1 chrX


Resizing and shifting genomic ranges can be useful in various contexts. For example:

- Narrowing peaks might help focus on the center of ChIP-seq binding sites.
- Shifting ranges can be used to look at regions adjacent to your features of interest. e.g. defining the predicted CRISPR cleavage site based on the position of the CRISPR gRNA sequence.

These operations demonstrate the flexibility of genomic range manipulations, which can be useful for fine-tuning analyses or testing hypotheses about the spatial relationships between genomic features.

## 5. Exercises

1. Calculate the average width of the ChIP-seq peaks on chromosome 22.
2. Determine how many peaks overlap with CpG islands.
3. Compute the percentage of promoter regions that have at least one overlapping ChIP-seq peak.

## Conclusion

In this tutorial, we've explored how to use BiocPy's genomic ranges functionality to perform various genomic analyses. These tools and techniques provide a powerful way to work with genomic interval data in Python, mirroring the capabilities from Bioconductor. They form the foundation for many more complex genomic analyses and can be applied to a wide range of biological questions.

```{note}
Refer to the [BiocPy documentation](https://biocpy.github.io/) for more detailed information on these packages and their functionalities.
```